In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
# Define values for rate constants (k)
k1 = 100000000       # kon for E + S to ES
k2 = 100       # koff for ES to E + S
k3 = 100000000       # kon for E + I to non-covalent EI
k4 = 10       # koff for EI to E + I
k5 = 0.01       # kon for EI to covalent EI*
k6 = 0.001       # koff for covalent EI* to non-covalent EI
kcat = 1    # kon ES to E+P

# Define concentrations at start of measurement t=0 (before addition of inhibitor or substrate)
E0 = 1*10**-12  #Unbound enzyme concentration E
ES0 = 0                 #Noncovalent enzyme-substrate complex ES
P0 = 0                   #Product
EI0 = 0                 #Non-covalent enzyme-inhibitor complex EI
EIi0 = 0                #Covalent enzyme-inhibitor complex (EI*) 

#Define assay constants
Is = [1*10**-6, 0.5*10**-6, 0.25*10**-6, 0.15*10**-6, 0.075*10**-6, 0.04*10**-6, 0.02*10**-6, 10*10**-9, 5*10**-9, 2.5*10**-9, 1.0*10**-9, 0*10**-9] # List of inhibitor concentrations. Is assumed to be constant, which is only true if inhibitor concentration is >10*E0.
S0 = 100*10**-9 # Substrate concentration. Is assumed to be constant, which is only true if substrate conversion is less than 10%
dt = 0.001 # Seconds per simulation step (dt). Minimum value for dt defined by k1*S0*dt < 1 and k3*I0*dt < 1

tframe =60 # Save a frame every 'tframe' seconds
tincub = 3600 # Preincubate with inhibitor for 'tincub' seconds
tsub = 1800 # Run with substrate for 'tsub' seconds

In [0]:
all_conc = []
all_t0 = []
all_t1 = []
for I in Is: # Loop over all I's
    all_conc.append(np.array([E0, ES0, P0, EI0, EIi0], dtype=np.double)) #matrix van concentraties
    # Voor toevoeging van S

    S=0 # Geen S in je systeem
        
                            # [         E,       ES, P,     EI, EI*]
    all_t0.append(np.array([[-k3*I-k1*S,  k2+kcat, 0,     k4,   0],  #E
                            [      k1*S, -k2-kcat, 0,      0,   0],  #ES
                            [         0,     kcat, 0,      0,   0],  #P
                            [      k3*I,        0, 0, -k4-k5,  k6],  #EI
                            [         0,        0, 0,     k5, -k6]]).T) #EI*

    # Change in E   = [E]*(-k3*I-k1*S) + [ES]*(k2+kcat)  + [P]*(0) + [EI]*(k4)     + [EI*]*(0)
    # Change in ES  = [E]*(k1*S)       + [ES]*(-k2-kcat) + [P]*(0) + [EI]*(0)      + [EI*]*(0)
    # Change in P   = [E]*(0)          + [ES]*(kcat)     + [P]*(0) + [EI]*(0)      + [EI*]*(0)
    # Change in EI  = [E]*(k3*I)       + [ES]*(0)        + [P]*(0) + [EI]*(-k4-k5) + [EI*]*(k6)
    # Change in EI* = [E]*(0)          + [ES]*(0)        + [P]*(0) + [EI]*(k5)     + [EI*]*(-k6)


    #Na toevoeging van S
    S= S0
    all_t1.append(np.array([[-k3*I-k1*S, k2+kcat, 0, k4, 0],
                            [k1*S, -k2-kcat, 0, 0, 0],
                            [0, kcat, 0, 0, 0],
                            [k3*I, 0, 0, -k4-k5, k6],
                            [0, 0, 0, k5, -k6]]).T)
    # Maak een lijst van kopien van de concentraties
concentraties = np.stack(all_conc)  
t0 = np.stack(all_t0)
t1 = np.stack(all_t1)                
plot = [concentraties.copy()]

tijdconc = concentraties.copy()
t = 0
# Voor toevoging van S (tincub/dt stappen = 30min)
for i in range(int(tincub/dt)):
    t+= 1
    tijdconc += np.dot(tijdconc, t0*dt).diagonal().T
    #tijdconc += [diff[i].T[i] for i in range(len(Is))] # Doe 1 dt tijdstap
    if (tijdconc < 0).any():
        raise ValueError("got a negative concentration, try decreasing dt")

    if t % int(tframe/dt)==0:
        plot.append(tijdconc.copy()) # Voeg de nieuwe concentraties toe aan de lijst

      # Na toevoeging van S (tsub/dt stappen)
for i in range(int(tsub/dt)):
    t += 1
    tijdconc += np.dot(tijdconc, t1*dt).diagonal().T   
    if (tijdconc < 0).any():
      raise ValueError("got a negative concentration, try decreasing dt")

    
    if t % int(tframe/dt)==0:
        plot.append(tijdconc.copy()) # Voeg de nieuwe concentraties toe aan de lijst

In [0]:
all_plots = np.array(plot).swapaxes(0,1) #swap ax 0 and ax 1

In [13]:
# This will make all the plots and dataframes
dataframes = []
fig, axes = plt.subplots(len(all_plots), 2, figsize=(12,len(all_plots)*6))
for n, plot in enumerate(all_plots):
  # Splits de lijst van figuren in E, ES, P, EI en EI*
  Eplot = [] 
  ESplot = []
  Pplot = []
  EIplot = []
  EIiplot = []
  EItot = []
  for frame in plot:
    Eplot.append(frame[0])
    ESplot.append(frame[1])
    Pplot.append(frame[2])
    EIplot.append(frame[3])
    EIiplot.append(frame[4])
    EItot.append(frame[3]+frame[4]) 
    
   # Plot de figuren
  labels = ('E', 'ES', "P", "EI", "EI*", "EI+Ei*")
  for i,y in enumerate((Eplot, ESplot, Pplot, EIplot, EIiplot, EItot)):
    axes[n,0].plot(y, label=labels[i])
  axes[n,0].legend()
  for i,y in enumerate((Eplot, ESplot, Pplot, EIplot, EIiplot, EItot)):
      axes[n,1].plot(y, label=labels[i])
  axes[n,1].legend()
  axes[n,1].set_ylim(0,E0) #zoom in tot E0
  axes[n,0].set_title("[I] = {}".format(str(Is[n])))
  axes[n,1].set_title("[I] = {}".format(str(Is[n])))
  
  df = pd.DataFrame(data={'E':Eplot,  "EI":EIplot, "EIi":EIiplot, "ES":ESplot, "P":Pplot})
  dataframes.append(df)
df = pd.concat(dataframes, keys=[I for I in Is], axis=1) # Combine all the dataframes

In [14]:
df


0.000001                              ... 0.000000                            
               E            EI           EIi  ...      EIi            ES             P
0   1.000000e-12  0.000000e+00  0.000000e+00  ...      0.0  0.000000e+00  0.000000e+00
1   5.372051e-14  5.371600e-13  4.091195e-13  ...      0.0  0.000000e+00  0.000000e+00
2   3.341377e-14  3.341131e-13  6.324731e-13  ...      0.0  0.000000e+00  0.000000e+00
3   2.232980e-14  2.232845e-13  7.543857e-13  ...      0.0  0.000000e+00  0.000000e+00
4   1.627986e-14  1.627912e-13  8.209289e-13  ...      0.0  0.000000e+00  0.000000e+00
..           ...           ...           ...  ...      ...           ...           ...
86  9.000980e-15  9.000980e-14  9.000980e-13  ...      0.0  9.009009e-14  1.405397e-10
87  9.000980e-15  9.000980e-14  9.000980e-13  ...      0.0  9.009009e-14  1.459451e-10
88  9.000980e-15  9.000980e-14  9.000980e-13  ...      0.0  9.009009e-14  1.513505e-10
89  9.000980e-15  9.000980e-14  9.000980e-13  ...      0.0  9.009009e-14  1.567559e-10
90  9.000980e-15  9.000980e-14  9.000980e-13  ...      0.0  9.009009e-14  1.621614e-10

[91 rows x 60 columns]

In [15]:
# Printing the values at timepoint
df.iloc[30]

1.000000e-06  E      9.009010e-15
              EI     9.009010e-14
              EIi    9.009009e-13
              ES     0.000000e+00
              P      0.000000e+00
5.000000e-07  E      1.785715e-14
              EI     8.928575e-14
              EIi    8.928571e-13
              ES     0.000000e+00
              P      0.000000e+00
2.500000e-07  E      3.508783e-14
              EI     8.771957e-14
              EIi    8.771926e-13
              ES     0.000000e+00
              P      0.000000e+00
1.500000e-07  E      5.714402e-14
              EI     8.571602e-14
              EIi    8.571400e-13
              ES     0.000000e+00
              P      0.000000e+00
7.500000e-08  E      1.081424e-13
              EI     8.110680e-14
              EIi    8.107508e-13
              ES     0.000000e+00
              P      0.000000e+00
4.000000e-08  E      1.856981e-13
              EI     7.427905e-14
              EIi    7.400228e-13
              ES     0.000000e+00
              

In [16]:
print(df.to_csv())

,1e-06,1e-06,1e-06,1e-06,1e-06,5e-07,5e-07,5e-07,5e-07,5e-07,2.5e-07,2.5e-07,2.5e-07,2.5e-07,2.5e-07,1.5e-07,1.5e-07,1.5e-07,1.5e-07,1.5e-07,7.5e-08,7.5e-08,7.5e-08,7.5e-08,7.5e-08,4e-08,4e-08,4e-08,4e-08,4e-08,2e-08,2e-08,2e-08,2e-08,2e-08,1e-08,1e-08,1e-08,1e-08,1e-08,5e-09,5e-09,5e-09,5e-09,5e-09,2.5e-09,2.5e-09,2.5e-09,2.5e-09,2.5e-09,1e-09,1e-09,1e-09,1e-09,1e-09,0.0,0.0,0.0,0.0,0.0
,E,EI,EIi,ES,P,E,EI,EIi,ES,P,E,EI,EIi,ES,P,E,EI,EIi,ES,P,E,EI,EIi,ES,P,E,EI,EIi,ES,P,E,EI,EIi,ES,P,E,EI,EIi,ES,P,E,EI,EIi,ES,P,E,EI,EIi,ES,P,E,EI,EIi,ES,P,E,EI,EIi,ES,P
0,1e-12,0.0,0.0,0.0,0.0,1e-12,0.0,0.0,0.0,0.0,1e-12,0.0,0.0,0.0,0.0,1e-12,0.0,0.0,0.0,0.0,1e-12,0.0,0.0,0.0,0.0,1e-12,0.0,0.0,0.0,0.0,1e-12,0.0,0.0,0.0,0.0,1e-12,0.0,0.0,0.0,0.0,1e-12,0.0,0.0,0.0,0.0,1e-12,0.0,0.0,0.0,0.0,1e-12,0.0,0.0,0.0,0.0,1e-12,0.0,0.0,0.0,0.0
1,5.372051494699701e-14,5.371600318687034e-13,4.09119453184323e-13,0.0,0.0,1.0288604221324701e-13,5.143508529649042e-13,3.827631048218589e-13,0.0,0.0,1.8892506127765085e-13,4